In [15]:
# %pip install transformers datasets evaluate
# %pip install transformers[torch]
# %pip install accelerate -U
%pip install bert_score


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.7 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import T5Tokenizer, T5Model
import numpy as np
import torch


In [3]:

train_data = load_dataset("wmt16", "de-en", split="train[:50000]")
val_data = load_dataset("wmt16", "de-en", split="validation")
test_data = load_dataset("wmt16", "de-en", split="test")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/4548885 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2169 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2999 [00:00<?, ? examples/s]

In [4]:
from transformers import AutoTokenizer

checkpoint = "google-t5/t5-small"
tokenizer_2c = AutoTokenizer.from_pretrained(checkpoint)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(DEVICE)
# model_name = "t5-small"
# # model_2c = T5ForConditionalGeneration.from_pretrained(model_name)
# tokenizer_2cc = T5Tokenizer.from_pretrained(model_name)

# model_2c = model_2c.to(DEVICE)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

cuda


In [5]:
source_lang = "de"
target_lang = "en"
prefix = "translate German to English: "

In [8]:
def preprocess_function_2c(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer_2c(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [9]:
tokenized_train_ = train_data.map(preprocess_function_2c, batched=True)


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [10]:
tokenized_val_ = val_data.map(preprocess_function_2c, batched=True)

Map:   0%|          | 0/2169 [00:00<?, ? examples/s]

In [11]:
from transformers import DataCollatorForSeq2Seq

data_collator_ = DataCollatorForSeq2Seq(tokenizer=tokenizer_2c, model=checkpoint)

In [12]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model_2c = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# model.save_pretrained("/t5small", from_pt=True)
# model = T5ForConditionalGeneration.from_pretrained("/home/my_name/Desktop/t5small")


In [13]:
training_args = Seq2SeqTrainingArguments(
    output_dir="model_2c",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
    logging_dir = './logs',
    logging_strategy = 'epoch'
)

trainer = Seq2SeqTrainer(
    model=model_2c,
    args=training_args,
    train_dataset=tokenized_train_,
    eval_dataset=tokenized_val_,
    tokenizer=tokenizer_2c,
    data_collator=data_collator_
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,2.520400,2.130549
2,2.346400,2.107708


TrainOutput(global_step=6250, training_loss=2.43339609375, metrics={'train_runtime': 798.2362, 'train_samples_per_second': 125.276, 'train_steps_per_second': 7.83, 'total_flos': 2687376416047104.0, 'train_loss': 2.43339609375, 'epoch': 2.0})

In [24]:
import evaluate as E

bleu = E.load("bleu")
meteor_metric = E.load("meteor")
bertscore_metric = E.load("bertscore")

def translate_2c(sentence):
    prefix = "translate German to English: "
    input_text = prefix + sentence
    input_ids = tokenizer_2c.encode(input_text, return_tensors="pt")
    input_ids = input_ids.to(DEVICE)
    output_ids = model_2c.generate(input_ids, max_length=100, num_beams=4, early_stopping=True)
    translated_sentence = tokenizer_2c.decode(output_ids[0], skip_special_tokens=True)
    return translated_sentence

def test_metrics_2c(data):
  references = []
  predictions = []
  for id, pair in enumerate(data):
      print(id,"/", len(data))

      tgt_sample = pair['translation']['en']
      src_sample = pair['translation']['de']

      predictions.append(translate_2c(src_sample))
      references.append([tgt_sample])

  bleu_score = bleu.compute(predictions=predictions, references=references)
  meteor_score = meteor_metric.compute(predictions=predictions, references=references)
  bertscore_score = bertscore_metric.compute(predictions=predictions, references=references, lang="en")

  bs_pre = np.mean(np.array(bertscore_score['precision']))
  bs_recall = np.mean(np.array(bertscore_score['recall']))
  bs_f1 = np.mean(np.array(bertscore_score['f1']))


  print(f'BLEU Score: {bleu_score}')
  print(f'METEOR Score: {meteor_score}')
  print(f'BERTScore-precision: {bs_pre}')
  print(f'BERTScore-recall: {bs_recall}')
  print(f'BERTScore-f1: {bs_f1}')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [28]:
model_2c.eval()
print("2C-Val Metrics")
test_metrics_2c(val_data)




2C-Val Metrics
0 / 2169
1 / 2169
2 / 2169
3 / 2169
4 / 2169
5 / 2169
6 / 2169
7 / 2169
8 / 2169
9 / 2169
10 / 2169
11 / 2169
12 / 2169
13 / 2169
14 / 2169
15 / 2169
16 / 2169
17 / 2169
18 / 2169
19 / 2169
20 / 2169
21 / 2169
22 / 2169
23 / 2169
24 / 2169
25 / 2169
26 / 2169
27 / 2169
28 / 2169
29 / 2169
30 / 2169
31 / 2169
32 / 2169
33 / 2169
34 / 2169
35 / 2169
36 / 2169
37 / 2169
38 / 2169
39 / 2169
40 / 2169
41 / 2169
42 / 2169
43 / 2169
44 / 2169
45 / 2169
46 / 2169
47 / 2169
48 / 2169
49 / 2169
50 / 2169
51 / 2169
52 / 2169
53 / 2169
54 / 2169
55 / 2169
56 / 2169
57 / 2169
58 / 2169
59 / 2169
60 / 2169
61 / 2169
62 / 2169
63 / 2169
64 / 2169
65 / 2169
66 / 2169
67 / 2169
68 / 2169
69 / 2169
70 / 2169
71 / 2169
72 / 2169
73 / 2169
74 / 2169
75 / 2169
76 / 2169
77 / 2169
78 / 2169
79 / 2169
80 / 2169
81 / 2169
82 / 2169
83 / 2169
84 / 2169
85 / 2169
86 / 2169
87 / 2169
88 / 2169
89 / 2169
90 / 2169
91 / 2169
92 / 2169
93 / 2169
94 / 2169
95 / 2169
96 / 2169
97 / 2169
98 / 2169
99 / 

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

BLEU Score: {'bleu': 0.03440728389860142, 'precisions': [0.18839903065875185, 0.045339873861247375, 0.01835494427084615, 0.008939012031286542], 'brevity_penalty': 1.0, 'length_ratio': 1.0162449164030727, 'translation_length': 44979, 'reference_length': 44260}
METEOR Score: {'meteor': 0.14124108208599095}
BERTScore-precision: 0.8040498145553251
BERTScore-recall: 0.784060718525165
BERTScore-f1: 0.7937898110991637


In [29]:
print("2C-Test Metrics")
test_metrics_2c(test_data)

2C-Test Metrics
0 / 2999
1 / 2999
2 / 2999
3 / 2999
4 / 2999
5 / 2999
6 / 2999
7 / 2999
8 / 2999
9 / 2999
10 / 2999
11 / 2999
12 / 2999
13 / 2999
14 / 2999
15 / 2999
16 / 2999
17 / 2999
18 / 2999
19 / 2999
20 / 2999
21 / 2999
22 / 2999
23 / 2999
24 / 2999
25 / 2999
26 / 2999
27 / 2999
28 / 2999
29 / 2999
30 / 2999
31 / 2999
32 / 2999
33 / 2999
34 / 2999
35 / 2999
36 / 2999
37 / 2999
38 / 2999
39 / 2999
40 / 2999
41 / 2999
42 / 2999
43 / 2999
44 / 2999
45 / 2999
46 / 2999
47 / 2999
48 / 2999
49 / 2999
50 / 2999
51 / 2999
52 / 2999
53 / 2999
54 / 2999
55 / 2999
56 / 2999
57 / 2999
58 / 2999
59 / 2999
60 / 2999
61 / 2999
62 / 2999
63 / 2999
64 / 2999
65 / 2999
66 / 2999
67 / 2999
68 / 2999
69 / 2999
70 / 2999
71 / 2999
72 / 2999
73 / 2999
74 / 2999
75 / 2999
76 / 2999
77 / 2999
78 / 2999
79 / 2999
80 / 2999
81 / 2999
82 / 2999
83 / 2999
84 / 2999
85 / 2999
86 / 2999
87 / 2999
88 / 2999
89 / 2999
90 / 2999
91 / 2999
92 / 2999
93 / 2999
94 / 2999
95 / 2999
96 / 2999
97 / 2999
98 / 2999
99 /

In [23]:
model_2c.save_pretrained("/content/drive/MyDrive/model_2c", from_pt=True)
new_model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/model_2c")


In [26]:
new_model = new_model.to(DEVICE)